In [38]:
import tweepy
import csv
import pandas as pd

from nltk.corpus import stopwords, reuters
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt
import datetime
all_tweets = []

In [58]:
def load_vaccine_csv():
    df = pd.read_csv('./covidvaccine.csv')
    df = df[['user_location','user_followers',	'user_friends',	'user_favourites',	'user_verified',	'date',	'text',	'source']]
    
    # '18-08-2020 12:55'
    # df.index = pd.to_datetime(df['date'])
    # for managebility we remove the hashtag, but it might have a future use-case
    # is_retweet is all False in this dataset
    # df = df[df['is_retweet'] == False]
    # df = df.dropna()
    return df

In [68]:
def try_parse(dateString):
    # return dateString
    # print(dateString)
    try:
        return datetime.datetime.strptime(dateString, '%Y-%m-%d %H:%M:%S')
    except Exception as e:
        try:
            return datetime.datetime.strptime(dateString, '%Y-%m-%d %H:%M')
        except Exception as e:
            try:
                return datetime.datetime.strptime(dateString, '%d-%m-%Y %H:%M')
            except Exception as e:
                return datetime.datetime.strptime(dateString, '%d-%m-%Y %H:%M:%S')


In [74]:
# 354390
# 354390
# 216910
df = load_vaccine_csv()
# df.index = pd.to_datetime(df.date, errors='coerce')

#18-08-2020 12:55
df = df[df.date.notnull()]
df = df[df.date.str.contains("-")]

df['date'] = df['date'].apply(lambda t: try_parse(t))
df['xxx'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M')
# df = df[df.date.isnull()]

df = df.sort_index()
df

,user_location,user_followers,user_friends,user_favourites,user_verified,date,text,source,xxx
0,Assam,64.0,11,110,False,2020-08-18 12:55:00,Australia to Manufacture Covid-19 Vaccine and ...,Twitter Web App,2020-08-18 12:55:00
1,NaN,1.0,17,0,False,2020-08-18 12:55:00,#CoronavirusVaccine #CoronaVaccine #CovidVacci...,Twitter for Android,2020-08-18 12:55:00
2,NaN,143.0,566,8,False,2020-08-18 12:46:00,Deaths due to COVID-19 in Affected Countries\n...,Twitter Web App,2020-08-18 12:46:00
3,NaN,29.0,25,620,False,2020-08-18 12:45:00,@Team_Subhashree @subhashreesotwe @iamrajchoco...,Twitter for Android,2020-08-18 12:45:00
4,"Adelaide, South Australia",83.0,497,10737,False,2020-08-18 12:45:00,@michellegrattan @ConversationEDU This is what...,Twitter Web App,2020-08-18 12:45:00
...,...,...,...,...,...,...,...,...,...
354385,"San Francisco, CA",0.0,83,0,False,2021-02-15 00:01:02,I called my lola to wish her a Happy Valentine...,Twitter for iPhone,2021-02-15 00:01:02
354386,Jakarta,2954.0,1023,2,False,2021-02-15 00:00:56,Cambodia’s Ministry of Health approves emergen...,SocialPilot.co,2021-02-15 00:00:56
354387,India,1771.0,270,65,False,2021-02-15 00:00:56,Cambodia’s Ministry of Health approves emergen...,SocialPilot.co,2021-02-15 00:00:56
354388,Pennsylvania,1191.0,44,44,False,2021-02-15 00:00:09,"Thanks for helping us get the word out, @abc27...",Twitter Web App,2021-02-15 00:00:09


In [ ]:
df2 = df.dropna()
df2

In [ ]:
load_vaccine_csv().dtypes


In [ ]:
# function to display data of each tweet 
def printtweetdata(n, ith_tweet): 
    print() 
    print(f"Tweet {n}:") 
    print(f"Username:{ith_tweet[0]}") 
    print(f"Description:{ith_tweet[1]}") 
    print(f"Location:{ith_tweet[2]}") 
    print(f"Following Count:{ith_tweet[3]}") 
    print(f"Follower Count:{ith_tweet[4]}") 
    print(f"Total Tweets:{ith_tweet[5]}") 
    print(f"Retweet Count:{ith_tweet[6]}") 
    print(f"Tweet Text:{ith_tweet[7]}") 
    print(f"Hashtags Used:{ith_tweet[8]}") 


In [ ]:
# function to perform data extraction 
def scrape(words, date_since, numtweet): 
      
    # Creating DataFrame using pandas 
    db = pd.DataFrame(columns=['username', 'description', 'location', 'following', 
                               'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags', 'created_at']) 
      
    # We are using .Cursor() to search through twitter for the required tweets. 
    # The number of tweets can be restricted using .items(number of tweets) 
    tweets = tweepy.Cursor(api.search, q=words + " -filter:retweets", lang="en", 
                           since=date_since, tweet_mode='extended').items(numtweet) 
     
    list_tweets = []
    while True:
        try:
            tweet = tweets.next()
            print(tweet)
            list_tweets.append(tweet)
            # Insert into db
        except tweepy.TweepError:
            time.sleep(60 * 15)
            continue
        except StopIteration:
            break
    # .Cursor() returns an iterable object. Each item in  
    # the iterator has various attributes that you can access to  
    # get information about each tweet 
    # list_tweets = [tweet for tweet in tweets] 
      
    # Counter to maintain Tweet Count 
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet 
    for tweet in list_tweets: 
        print(tweet)
        username = tweet.user.screen_name 
        description = tweet.user.description 
        location = tweet.user.location 
        following = tweet.user.friends_count 
        followers = tweet.user.followers_count 
        totaltweets = tweet.user.statuses_count 
        retweetcount = tweet.retweet_count 
        created_at = tweet.created_at
        hashtags = tweet.entities['hashtags'] 
          
        # Retweets can be distinguished by a retweeted_status attribute, 
        # in case it is an invalid reference, except block will be executed 
        try: 
            text = tweet.retweeted_status.full_text 
            all_tweets.append(text)
            # print(text)
        except AttributeError: 
            text = tweet.full_text 
        hashtext = list() 
        for j in range(0, len(hashtags)): 
            hashtext.append(hashtags[j]['text']) 
          
        # Here we are appending all the extracted information in the DataFrame 
        ith_tweet = [username, description, location, following, 
                     followers, totaltweets, retweetcount, text, hashtext, created_at] 
        db.loc[len(db)] = ith_tweet 
          
        # Function call to print tweet data on screen 
        # printtweetdata(i, ith_tweet) 
        i = i+1
    filename = 'scraped_tweets.csv'
      
    # we will save our database as a CSV file. 
    db.to_csv(filename) 
    return db

In [ ]:
 # Enter your own credentials obtained  
    # from your developer account 
consumer_key = "SCOPBRKeG4nRCEa7XkMoqQ"
consumer_secret = "RYInMkLiNyg0iKC3g89Y0f8g8kbNFSsNjZXNYBYILQ"
access_key = "107275871-PtAa9t7OJX82IEbGjdOzaFMgeCWQyU8haNPhN4mD"
access_secret = "Q691rTrztjXeYAP5FEA5kZWRhuFRtH8sq6msPgUeD1TzI"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_key, access_secret) 
api = tweepy.API(auth) 
    
# Enter Hashtag and initial date 
# print("Enter Twitter HashTag to search for") 
words = 'covid19vaccine' 
# print("Enter Date since The Tweets are required in yyyy-mm--dd") 
date_since = '2016-01-01' 
    
# number of tweets you want to extract in one run 
numtweet = 1000  
df = scrape(words, date_since, numtweet) 
print('Scraping has completed!') 

In [ ]:

df.describe()
# ####input your credentials here
# consumer_key = 'SCOPBRKeG4nRCEa7XkMoqQ'
# consumer_secret = 'RYInMkLiNyg0iKC3g89Y0f8g8kbNFSsNjZXNYBYILQ'
# access_token = '107275871-PtAa9t7OJX82IEbGjdOzaFMgeCWQyU8haNPhN4mD'
# access_token_secret = 'Q691rTrztjXeYAP5FEA5kZWRhuFRtH8sq6msPgUeD1TzI'

# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth,wait_on_rate_limit=True)
# #####United Airlines
# # Open/Create a file to append data
# csvFile = open('ua.csv', 'a')
# #Use csv Writer
# csvWriter = csv.writer(csvFile)

# all_tweets = []

# for tweet in tweepy.Cursor(api.search,q="#dulux",count=100,
#                            lang="en",
#                            since="2015-11-03").items():
#     print (tweet.created_at, tweet.text)
#     all_tweets.append(tweet.text)
#     csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

In [ ]:
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

In [ ]:
def process_text(doc):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', doc)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return ' '.join(output)

In [ ]:
all_tweets
big_string = ' '.join(all_tweets)
input_text = process_text(big_string)

In [ ]:
wc = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(input_text)
# plt.imshow(wc)
plt.figure(figsize=(100,20))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
def process_text_bg(doc):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', doc)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    sw_words = [word.lower() for word in lem if word.lower() not in sw]
    bigrams = ngrams(sw_words, 2)
    output = ['_'.join(i) for i in bigrams]
    return ' '.join(output)

In [ ]:
from nltk.util import ngrams
input_bigrams = process_text_bg(big_string)
wc = WordCloud().generate(input_bigrams)
plt.figure(figsize=(15,24))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
all_tweets

In [ ]:
db